In [2]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00


In [1]:
from peewee import *
# Connect to a MySQL database on network.
#Base de datos
Server = '23.102.103.53'
Colección = 'BrayanMontenegro'
Usuario = 'bmontenegroprueba'
Contraseña = 'bmontenegroprueba'
port_ok = True
port_cont = 2105
while port_ok:
    try:
        mysql_db = MySQLDatabase('mysql', user=Usuario, password=Contraseña,
                                host=Server, port=port_cont)
        mysql_db.connect()
        print(port_cont)
        port_ok = False  
    except:
        port_cont = port_cont +1
        port_ok = True    
        print("error ",port_cont)

error  2106
error  2107


In [8]:
import pyodbc
import pandas as pd
Server = '23.102.103.53'
Colección = 'BrayanMontenegro'
Usuario = 'bmontenegroprueba'
Contraseña = 'bmontenegroprueba'
conexion = pyodbc.connect('DRIVER={ODBC DRIVER 17 for SQL server};SERVER='+Server+';DATABASE='+Colección+';UID='+Usuario+';PWD='+Contraseña)
conexion



ImportError: dlopen(/opt/homebrew/lib/python3.11/site-packages/pyodbc.cpython-311-darwin.so, 0x0002): symbol not found in flat namespace '_SQLAllocHandle'

In [7]:
import _mssql

server = 'SERVER_NAME'
user = 'USER_NAME'
password = '<PASSWORD>'
database = 'MY_DATABASE'
conn = _mssql.connect(server, user, password, database)

ModuleNotFoundError: No module named '_mssql'

In [2]:
mysql_db.close()

False